In [1]:
중구 = [1114010100,1114010200,1114010300,1114010400,1114010500,1114010600,1114010700,1114010800,1114010900,1114011000,1114011100,1114011200,1114011300,1114011400,1114011500,1114011600,1114011700,1114011800,1114011900,1114012000,1114012100,1114012200,1114012300,1114012400,1114012500,1114012600,1114012700,1114012800,1114012900,1114013000,1114013100,1114013200,1114013300,1114013400,1114013500,1114013600,1114013700,1114013800,1114013900,1114014000,1114014100,1114014200,1114014300,1114014400,1114014500,1114014600,1114014700,1114014800,1114014900,1114015000,1114015100,1114015200,1114015300,1114015400,1114015500,1114015600,1114015700,1114015800,1114015900,1114016000,1114016100,1114016200,1114016300,1114016400,1114016500,1114016600,1114016700,1114016800,1114016900,1114017000,1114017100,1114017200,1114017300,1114017400]
jung = ['무교동','다동','태평로1가','을지로1가','을지로2가','남대문로1가','삼각동','수하동','장교동','수표동','소공동','남창동','북창동','태평로2가','남대문로2가','남대문로3가','남대문로4가','남대문로5가','봉래동1가','봉래동2가','회현동1가','회현동2가','회현동3가','충무로1가','충무로2가','명동1가','명동2가','남산동1가','남산동2가','남산동3가','저동1가','충무로4가','충무로5가','인현동2가','예관동','묵정동','필동1가','필동2가','필동3가','남학동','주자동','예장동','장충동1가','장충동2가','광희동1가','광희동2가','쌍림동','을지로6가','을지로7가','을지로4가','을지로5가','주교동','방산동','오장동','을지로3가','입정동','산림동','충무로3가','초동','인현동1가','저동2가','신당동','흥인동','무학동','황학동','서소문동','정동','순화동','의주로1가','충정로1가','중림동','의주로2가','만리동1가','만리동2가']

In [2]:
import requests #필요한 모듈들을 import 합니다.
from bs4 import BeautifulSoup
import pandas as pd

names = []
areas = []
prices = []
area_per_price = []
for area_code in 중구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
    
    names.append(new_name) # lists in lists 로 저장합니다
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(jung)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)

kk = {'dong':중구,'per price':amean}
df = pd.DataFrame(kk,index = jung)
df

,dong,per price
무교동,1114010100,0
다동,1114010200,0
태평로1가,1114010300,0
을지로1가,1114010400,0
을지로2가,1114010500,16950000
남대문로1가,1114010600,0
삼각동,1114010700,0
수하동,1114010800,0
장교동,1114010900,0
수표동,1114011000,0


In [3]:
df.to_csv('junggu.csv')